# RWI-GEO-RED Panel Data Preprocessing: Household Purchasing Power (HK)

## About the Dataset
Holds real estate listings for the 15 bigggest cities in germany from 2007 to 2024



### Dataset: CampusFile_WK_cities.csv

### Data Structure
- **Temporal Coverage**: Panel data across multiple years
- **Geographic Coverage**: 15 largest cities in Germany

## Setup: Load Required Libraries

In [56]:
library(dplyr)
library(ggplot2)
library(tidyr)

# Load Hauskauf data

In [57]:
raw_data <- read.csv('data/RWI-GEO-RED/panel-15-largest-cities-germany/CampusFile_WK_cities.csv')

In [58]:
df <- raw_data
total_rows_in_raw_data = nrow(df)
total_rows_in_raw_data

[1] 828135

# Preprocess the data

In [59]:
summary(df)

      obid                plz          kaufpreis         heizkosten     
 Min.   :  6745100   Min.   :   -9   Min.   :  18800   Min.   : -9.000  
 1st Qu.: 60035976   1st Qu.:13127   1st Qu.: 120000   1st Qu.: -9.000  
 Median : 84545422   Median :40474   Median : 227000   Median : -8.000  
 Mean   : 89317259   Mean   :38731   Mean   : 290077   Mean   : -8.482  
 3rd Qu.:117020217   3rd Qu.:60322   3rd Qu.: 389000   3rd Qu.: -8.000  
 Max.   :156377603   Max.   :98000   Max.   :1699000   Max.   :300.000  
    baujahr     letzte_modernisierung  wohnflaeche     grundstuecksflaeche
 Min.   :1500   Min.   :  -9.0        Min.   : 24.01   Min.   :-8         
 1st Qu.:1950   1st Qu.:  -9.0        1st Qu.: 59.65   1st Qu.:-8         
 Median :1972   Median :  -9.0        Median : 77.00   Median :-8         
 Mean   :1968   Mean   : 355.1        Mean   : 83.34   Mean   :-8         
 3rd Qu.:2000   3rd Qu.:  -9.0        3rd Qu.:100.70   3rd Qu.:-8         
 Max.   :2024   Max.   :2018.0        M

## Range of the house inserate

In [60]:
# adat -> the date of the inserat
df %>%
    summarise(
        min_adat = min(as.Date(paste0(adat, "-01")), na.rm = TRUE),
        max_adat = max(as.Date(paste0(adat, "-01")), na.rm = TRUE)
    )

min_adat,max_adat
<date>,<date>
2007-01-01,2024-12-01


data goes from 2007 to 2024

## Extract only berlin samples

We only want to createt a model of the berlin real estate listings

In [61]:
extract_berlin_samples <- function(source_df) {
    df_berlin <- subset(source_df, !is.na(plz) &
        as.integer(as.character(plz)) >= 10115 &
        as.integer(as.character(plz)) <= 14199)
    return (
        df_berlin
    )
}

In [62]:
dfb = extract_berlin_samples(df)

In [63]:
cat("Total rows in raw data:", total_rows_in_raw_data, "\n")
cat("Rows in Berlin sample:", nrow(dfb), "\n")
cat("Percentage of Berlin rows:", round(100 * nrow(dfb) / total_rows_in_raw_data, 2), "%\n")

Total rows in raw data: 828135 
Rows in Berlin sample: 180409 
Percentage of Berlin rows: 21.78 %


## Sentinel cleanup

The data is full of placeholder integer values that symbolise missing data

- -9 -> missing data
- -8 -> did not answer

etc ...
 
these need to be cleaned up

In [64]:
# util function for replacing sentinel values with NA
sentinels <- c(-9, -8, -7, -6, -5, -2)
replace_sentinels <- function(x, s = sentinels) {
  x[x %in% s] <- NA
  return (
    x
  )
}

In [65]:
c(colnames(df))

[1] "obid"                   "plz"                    "kaufpreis"             
 [4] "heizkosten"             "baujahr"                "letzte_modernisierung" 
 [7] "wohnflaeche"            "grundstuecksflaeche"    "nutzflaeche"           
[10] "etage"                  "anzahletagen"           "zimmeranzahl"          
[13] "nebenraeume"            "schlafzimmer"           "badezimmer"            
[16] "parkplatzpreis"         "wohngeld"               "ev_kennwert"           
[19] "laufzeittage"           "immobilientyp"          "aufzug"                
[22] "balkon"                 "denkmalobjekt"          "einbaukueche"          
[25] "einliegerwohnung"       "ev_wwenthalten"         "gaestewc"              
[28] "garten"                 "keller"                 "parkplatz"             
[31] "rollstuhlgerecht"       "bauphase"               "ausstattung"           
[34] "energieeffizienzklasse" "energieausweistyp"      "haustier_erlaubt"      
[37] "heizungsart"            "kategorie_Wohnung"      "objektzustand"         
[40] "ergg_1km"               "blid"                   "gid2019"               
[43] "kid2019"                "bef1"                   "bef2"                  
[46] "bef10"                  "bef9"                   "bef8"                  
[49] "bef7"                   "bef6"                   "bef5"                  
[52] "bef4"                   "bef3"                   "anbieter"              
[55] "spell"                  "dataset_version"        "adat"                  
[58] "edat"                   "price_sqm"

In [66]:
cols_with_sentinels <- c(colnames(df))
df[cols_with_sentinels] <- lapply(df[cols_with_sentinels], replace_sentinels)

## Datatypes: Factor columns 

In [67]:
factor_cols <- c(
    "plz",
    "immobilientyp",
    "parkplatz",
    "balkon",
    "denkmalobjekt",
    "einbaukueche",
    "gaestewc",
    "garten",
    "keller",
    "rollstuhlgerecht",
    "ausstattung",
    "energieausweistyp",
    "kategorie_Wohnung",
    "dataset_version"
)
df[factor_cols] <- lapply((df[factor_cols]), as.factor)

## Special cases

- obid as character (object id)
- adata edat (start end of listing) need their information extracted from being a string "YYYY-MM"

In [68]:
df$obid <- as.character(df$obid)

## Feature ADAT EDAT: Start of listing end of listing

Here i need to extract the date information from the string and ill also add a time on the market feature


 
adat and edat stand for start and end of listing

These columns are sometimes formatted as `"YYYY-MM"` (year and month only) and sometimes as `"YYYY-MM-DD"` (full date).  

For consistency and analysis, we extract the year and month components, convert them to proper date objects, and calculate the "time on market"

In [69]:
extract_year_month <- function(date_str) {
    # Handles "YYYY-MM" and "YYYY-MM-DD", and converts factors to character
    if (is.na(date_str)) return(list(year=NA_integer_, month=NA_integer_))
    
    # Convert to character if it's a factor
    date_str <- as.character(date_str)
    
    parts <- unlist(strsplit(date_str, "-"))
    if (length(parts) >= 2) {
        year <- as.integer(parts[1])
        month <- as.integer(parts[2])
        return(list(year=year, month=month))
    } else {
        return(list(year=NA_integer_, month=NA_integer_))
    }
}

In [70]:
# Check the format of adat and edat columns
cat("Class of adat:", class(df$adat), "\n")
cat("Class of edat:", class(df$edat), "\n")
cat("Sample adat values:\n")
head(df$adat, 10)
cat("\nSample edat values:\n")
head(df$edat, 10)

Class of adat: character 
Class of edat: character 
Sample adat values:


[1] "2007-1"  "2007-9"  "2007-5"  "2007-5"  "2007-1"  "2007-5"  "2007-1" 
 [8] "2007-11" "2007-1"  "2007-6"


Sample edat values:


[1] "2007-4"  "2007-12" "2007-10" "2007-10" "2007-8"  "2007-10" "2007-1" 
 [8] "2007-12" "2007-8"  "2007-6"

In [71]:
# Apply extract_year_month to adat and edat columns
adat_ym <- lapply(df$adat, extract_year_month)
df$listing_start_year <- sapply(adat_ym, function(x) x$year)
df$listing_start_month <- sapply(adat_ym, function(x) x$month)

edat_ym <- lapply(df$edat, extract_year_month)
df$listing_end_year <- sapply(edat_ym, function(x) x$year)
df$listing_end_month <- sapply(edat_ym, function(x) x$month)

In [72]:
# Calculate time on market in months
df$time_on_market_months <- (df$listing_end_year - df$listing_start_year) * 12 + (df$listing_end_month - df$listing_start_month)

In [73]:
df %>% select(adat, edat, listing_start_year, listing_start_month, listing_end_year, listing_end_month, time_on_market_months) %>% head(10)

,adat,edat,listing_start_year,listing_start_month,listing_end_year,listing_end_month,time_on_market_months
,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>
1,2007-1,2007-4,2007,1,2007,4,3
2,2007-9,2007-12,2007,9,2007,12,3
3,2007-5,2007-10,2007,5,2007,10,5
4,2007-5,2007-10,2007,5,2007,10,5
5,2007-1,2007-8,2007,1,2007,8,7
6,2007-5,2007-10,2007,5,2007,10,5
7,2007-1,2007-1,2007,1,2007,1,0
8,2007-11,2007-12,2007,11,2007,12,1
9,2007-1,2007-8,2007,1,2007,8,7


# Drop unneccessary columns

In [74]:
cols_to_drop = c(
    "grundstuecksflaeche", # all values are NA
    "einliegerwohnung", # all values are NA,
    "bef6", # only has about 10 rows with values
    "bef7","bef8","bef9","bef10", # no values in these columns,
    "haustier_erlaubt", # no values here
    "bauphase", # no values for wohnungen
    "nebenräume" # all values are 1 no information here
)
df <- df %>% select(-cols_to_drop)

ERROR: [1m[33mError[39m in `select()`:[22m
[33m![39m Can't select columns that don't exist.
[31m✖[39m Column `nebenräume` doesn't exist.


# Result

We now have a preprocessed dataset of only berlin listings with less columns, removed sentinels and some basic feature cleanup

In [ ]:
summary(df)

     obid                plz           kaufpreis         heizkosten    
 Length:828135      10245  :  6407   Min.   :  18800   Min.   :  0.00  
 Class :character   10115  :  5936   1st Qu.: 120000   1st Qu.: 40.00  
 Mode  :character   10247  :  5346   Median : 227000   Median : 60.00  
                    50858  :  4466   Mean   : 290077   Mean   : 74.19  
                    60486  :  4369   3rd Qu.: 389000   3rd Qu.: 94.50  
                    (Other):801154   Max.   :1699000   Max.   :300.00  
                    NA's   :   457                     NA's   :827988  
    baujahr     letzte_modernisierung  wohnflaeche      nutzflaeche     
 Min.   :1500   Min.   :1890          Min.   : 24.01   Min.   :   0.00  
 1st Qu.:1950   1st Qu.:2005          1st Qu.: 59.65   1st Qu.:   6.00  
 Median :1972   Median :2010          Median : 77.00   Median :  14.00  
 Mean   :1968   Mean   :2008          Mean   : 83.34   Mean   :  39.95  
 3rd Qu.:2000   3rd Qu.:2014          3rd Qu.:100.70   3rd 

In [75]:
write.csv(df, file = "panel-berlin-WK-preprocessed.csv", row.names = FALSE)